<a href="https://colab.research.google.com/github/mshilpaa/ethereum-data-analysis/blob/master/Malicious_Address_Prediction/Queries_for_malicious_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Queries for Google Big Query

**Full join in and out transactions**
```
SELECT out_txn.address as address1, in_txn.address as address2, out_txn.no_of_out_transactions as no_of_out_transactions, out_txn.tot_ether_sent as tot_ether_sent, in_txn.no_of_in_transactions as no_of_in_transactions, in_txn.tot_ether_recieved as tot_ether_recieved FROM adroit-medium-269914.ethereum.in_txn as in_txn
FULL JOIN adroit-medium-269914.ethereum.out_txn as out_txn ON in_txn.address = out_txn.address
```

**Tokens sent and recieved**
```
SELECT token_recd.address as address1, token_sent.address as address2, token_recd.tot_distinct_tokens_recieved, token_recd.tot_tokens_recieved, token_recd.tot_token_value_recieved,
token_sent.tot_distinct_tokens_sent, token_sent.tot_tokens_sent, token_sent.tot_token_value_sent 
FROM (SELECT to_address as address, count(distinct token_address) as tot_distinct_tokens_recieved, count(*) as tot_tokens_recieved, sum(safe_cast(value as FLOAT64)) as tot_token_value_recieved FROM `bigquery-public-data.crypto_ethereum.token_transfers` WHERE to_address in (SELECT address FROM `adroit-medium-269914.ethereum.addrlist`)group by to_address) as token_recd 
FULL JOIN 
(SELECT from_address as address, count(distinct token_address) as tot_distinct_tokens_sent, count(*) as tot_tokens_sent, sum(safe_cast(value as FLOAT64)) as tot_token_value_sent FROM `bigquery-public-data.crypto_ethereum.token_transfers` WHERE from_address in (SELECT address FROM `adroit-medium-269914.ethereum.addrlist`) group by from_address) as token_sent
ON token_recd.address = token_sent.address
```

**Monthly txn:**
```
select m_in.address as address1, m_out.address as address2, m_in.monthly_out_txn, m_out.monthly_in_txn
from (select from_address as address, sum(number_tx_sent)/count(month) as monthly_out_txn 
from  
(with traces_clean as (
    select * from `bigquery-public-data.crypto_ethereum.transactions` 
    
)
select 
        TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC') as month, 
        from_address,
        count(*) as number_tx_sent
    from traces_clean
    where from_address in (SELECT address FROM `adroit-medium-269914.ethereum.addrlist`)
    group by TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC'), from_address)
group by from_address) as m_in
full join
(select to_address as address, sum(number_tx_received)/count(month) as monthly_in_txn 
from 
(with traces_clean as (
    select * from `bigquery-public-data.crypto_ethereum.transactions` 
    
)
    select 
        TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC') as month, 
        to_address,
        count(*) as number_tx_received
    from traces_clean
    where to_address in (SELECT address FROM `adroit-medium-269914.ethereum.addrlist`)
    group by TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC'), to_address)
group by to_address) as m_out
on m_in.address = m_out.address
```

**Time b/w txn:**
```
select time_out.address as address1, time_out.no_of_in_txn, time_out.total_count,time_in.address as address2, time_in.no_of_out_txn, time_in.total_out_count,
from
(SELECT to_address as address, TIMESTAMP_DIFF(max(block_timestamp), min(block_timestamp),second)  as no_of_in_txn, count(block_timestamp) as total_count, FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE to_address in (SELECT address FROM `adroit-medium-269914.ethereum.addrlist`) group by to_address) as time_out
full join
(SELECT from_address as address, TIMESTAMP_DIFF(max(block_timestamp), min(block_timestamp),second)  as no_of_out_txn, count(block_timestamp) as total_out_count, FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE from_address in (SELECT address FROM `adroit-medium-269914.ethereum.addrlist`) group by from_address) as time_in
on time_out.address = time_in.address
```